<a href="https://colab.research.google.com/github/mebirtukan/Event_detection/blob/main/LSTMCONV2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from scipy.io import loadmat
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv1D, MaxPool1D, Conv2D, ConvLSTM2D,MaxPool2D, MaxPooling2D, ConvLSTM3D, LSTM
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, cohen_kappa_score



In [ ]:
!wget www.kasprowski.pl/datasets/events.zip
!unzip events.zip > xx

--2022-07-25 12:57:58--  http://www.kasprowski.pl/datasets/events.zip
Resolving www.kasprowski.pl (www.kasprowski.pl)... 94.152.164.213
Connecting to www.kasprowski.pl (www.kasprowski.pl)|94.152.164.213|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2909538 (2.8M) [application/zip]
Saving to: ‘events.zip.1’

events.zip.1        100%[===================>]   2.77M  2.68MB/s    in 1.0s    

2022-07-25 12:57:59 (2.68 MB/s) - ‘events.zip.1’ saved [2909538/2909538]

replace data/TH20_trial1_labelled_MN.mat? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
def openfile(filename):
  mat = loadmat(filename)
  mdata = mat['ETdata']
  mtype = mdata.dtype
  ndata = {n: mdata[n][0,0] for n in mtype.names}
  ndata

  data_headline = ndata['pos']
  print(data_headline)
  data_headline = data_headline[0]
  data_raw = ndata['pos']
  len(data_raw)
  pdata = pd.DataFrame(data_raw,columns=data_headline)
  print(pdata)
  df=pd.DataFrame(pdata)
  df=pd.DataFrame(pdata)
  df.dtypes
  df[1.0]=df[1.0].astype(int)
  df[1.0].dtypes
  x=pdata.iloc[:, 3:5].values
  y=pdata.iloc[:, 5].values
  x[0]
  x.shape[0]
  return x ,y

In [ ]:


def open_list_of_files(files_to_load):
  samples = []
  labels =[]
  for my_file in files_to_load:
    sam,lab = openfile(my_file)
    #ssam,slab = make_sequences(sam,lab,50)
    print('Number of samples so far:',len(samples))
    samples.extend(sam)
    labels.extend(lab)
  samples = np.array(samples)
  labels = np.array(labels)
  print('Number of samples at the end:',len(samples))
  return samples,labels
files_to_load = ['data/UH21_img_Rome_labelled_MN.mat','data/TH34_img_Europe_labelled_MN.mat', 'data/UH47_img_Europe_labelled_RA.mat' ,'data/UH33_img_vy_labelled_MN.mat', 'data/UH21_img_Rome_labelled_RA.mat', 'data/UH27_img_vy_labelled_MN.mat', 'data/UH47_img_Europe_labelled_MN.mat']

x,y = open_list_of_files(files_to_load)
print(x.shape)
print(y.shape)

[[6.78053517e+09 2.20000000e+01 2.30000000e+01 5.53437900e+02
  4.12084800e+02 1.00000000e+00]
 [6.78053717e+09 2.20000000e+01 2.30000000e+01 5.54019700e+02
  4.12477500e+02 1.00000000e+00]
 [6.78053917e+09 2.20000000e+01 2.30000000e+01 5.54564700e+02
  4.12221400e+02 1.00000000e+00]
 ...
 [6.79050722e+09 2.50000000e+01 2.80000000e+01 4.88114900e+02
  6.36379100e+02 1.00000000e+00]
 [6.79050922e+09 2.50000000e+01 2.70000000e+01 4.88146100e+02
  6.36474000e+02 1.00000000e+00]
 [6.79051122e+09 2.50000000e+01 2.80000000e+01 4.89047300e+02
  6.36165000e+02 1.00000000e+00]]
      6.780535e+09  2.200000e+01  2.300000e+01  5.534379e+02  4.120848e+02  \
0     6.780535e+09          22.0          23.0      553.4379      412.0848   
1     6.780537e+09          22.0          23.0      554.0197      412.4775   
2     6.780539e+09          22.0          23.0      554.5647      412.2214   
3     6.780541e+09          22.0          23.0      555.3923      411.9315   
4     6.780543e+09          22.0  

In [ ]:
#s=pd.DataFrame(s)

In [ ]:
# sequence 
def make_sequences(samples, labels, sequence_dim = 100, sequence_lag = 1, sequence_attributes = 2):
    nsamples = []
    nlabels = [] 
    for i in range(0,samples.shape[0]-sequence_dim,sequence_lag):
            nsample = np.zeros((sequence_dim,sequence_attributes))
            for j in range(i,i+sequence_dim):
                nsample[j-i,0] = samples[j,0]
                nsample[j-i,1] = samples[j,1]
            nlabel = labels[i+sequence_dim//2]
            #print("Sample",nsample)
            #print("Label",nlabel)
            nsamples.append(nsample)
            nlabels.append(nlabel)
        
    #samples = np.array(nsamples)
    #labels = np.array(nlabels)
    return samples,labels 

In [ ]:
#s=pd.DataFrame(s)

In [ ]:
#x,y=openfile('data/TH34_img_Europe_labelled_MN.mat')
X

array([[0.5818, 0.3927],
       [0.545 , 0.2561],
       [0.8276, 0.2899],
       ...,
       [0.1553, 0.0876],
       [0.1967, 0.0172],
       [0.3833, 0.3778]])

In [ ]:
#X=list(X)

In [ ]:
#X

In [ ]:
#np.array(x)

In [ ]:
s.shape

(28934, 2)

In [ ]:
#x1,y1=openfile('data/UH33_img_vy_labelled_MN.mat')


In [ ]:
import csv
import numpy as np

def caluculate_velocity(data):
  times = data[:,[0]]
  ts = []

  # convert time to second
  for t in times:
    ts.append(float(t)/1000.0)
  
  Xs = data[:,[0]]
  Ys = data[:,[1]]

  diffX = [] #x values difference
  diffY = [] #y values difference 
  diffT = [] #time values difference

  # difX=x2-x1 and diffY=y2-y1
  for i in range(len(data) - 1):
    diffX.append(float(Xs[i+1]) - float(Xs[i]) )
    diffY.append(float(Ys[i+1]) - float(Ys[i]) )
    diffT.append(float(ts[i+1]) - float(ts[i]) )
  
  # distance = sqrt(x^2 + y^2)
    distancex=diffX
    distancey=diffY
  #distancexx = np.sqrt(np.power(diffX,2) + np.power(diffY,2))
  #distanceyy = np.sqrt(np.power(diffX,2) + np.power(diffY,2))

  #print('distance=', distance)
  #velocityx = np.divide(distancex,diffT)
  #velocityy = np.divide(distancey,diffT)
  velocityx=distancex
  velocityy=distancey

  #velocity_x = []
  #velocity_y=[]

  # calculate velocity
  #vel_X=velocityx
  #vel_Y=velocityy
  #velocity = np.divide(distance,diffT)
  velocityX = np.absolute(velocityx)
  velocityY = np.absolute(velocityy)
  

  #velocity = distance
  #velocity = diffX+diffY
  #velocity=np.absolute((velocityX, velocityY))
  #velocity = np.absolute(velocity)
  #print('velocity=', velocity)
  #print(('minimumu_velocity=',min(velocity)))
  #print(('maximumu_velocity=',max(velocity)))

 # for v in velocity:
 #   print(v,v_threshold,v<v_threshold)
  print('Time=', (diffT))
  print('velocitX=', velocityX)
  print('velocitY=', velocityY)

  #mvmts = []
 
  #store 1 in mvmts[] if velocity is less than threshold else store 2

  return velocityX, velocityY
  #print(velocity_x, velocity_y)





In [ ]:
Y.shape, y.shape

((28929, 3), (28934,))

In [ ]:
vx,vy=caluculate_velocity(x)



print(len(vx))
print(vx[0])
print(np.array(vx).shape)
print(np.array(vy).shape)

y = np.array(Y[:-1])
com = np.vstack((vx,vy)).T


Time= [0.0005817999999998547, 0.0005450000000001287, 0.0008275999999999284, 0.00016710000000008662, -0.00028980000000011774, -0.0005602000000000107, -0.0002932999999998298, 0.0004881999999999387, 0.0014286999999999495, 0.0017715999999999843, -1.8499999999921357e-05, 7.129999999999637e-05, 0.00039719999999998645, 0.00039450000000007535, 0.0006600999999999413, 0.00035759999999995795, 1.5500000000057135e-05, 0.00021129999999991433, 0.0005252000000000034, 8.390000000002562e-05, -0.0001342999999999206, -0.00010019999999999474, 0.0003109999999999502, 0.0003634000000000137, -5.500000000002725e-05, -0.00016779999999994022, -0.00033260000000012724, 0.00022070000000007362, 0.00040509999999993607, -0.00023139999999999272, -0.00016970000000005037, 4.4000000000155026e-05, -0.00032530000000008386, -5.430000000006263e-05, 4.300000000001525e-05, -0.0004685999999999302, -0.0004994000000000387, -0.0003391000000000366, 2.0000000000020002e-05, 0.00030770000000002184, 0.00020770000000014388, -6.36000000001

In [ ]:
y = np.array(y[:-1])
#y1 = np.array(y1[:-1])

In [ ]:
com = np.vstack((vx,vy)).T
#com1 = np.vstack((vx1,vy1)).T

In [ ]:
sequence_dim = 20
print("Samples shape before sequencing",com.shape)

print("Converting to sequences of length {}".format(sequence_dim))
X, Y = make_sequences(com, y, sequence_dim)
print("Samples shape after sequencing: {}".format(X.shape))
print("Labels shape after sequencing: {}".format(Y.shape))


Samples shape before sequencing (28933, 2)
Converting to sequences of length 20
Samples shape after sequencing: (28933, 2)
Labels shape after sequencing: (28927, 3)


In [ ]:

#print("Samples shape before sequencing",com1.shape)
#print("Converting to sequences of length {}".format(sequence_dim))
#x1,y1 = make_sequences(com1, y1, sequence_dim)
#print("Samples shape after sequencing: {}".format(x1.shape))
#print("Labels shape after sequencing: {}".format(y1.shape))

In [ ]:
#x1=np.reshape(x1,(4938, 50,2,1))

In [ ]:
#x=np.reshape(x,(4938, 50,2,1))

In [ ]:
#x1.shape, x.shape

In [ ]:
from tensorflow.keras.layers import BatchNormalization
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
lb.fit(Y)
Y = lb.transform(Y)
#y1 = lb.transform(y1)


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
models = keras.Sequential()
models.add(keras.Input(shape=(20, 2,1)))

#intializing randomly orthogonal weights.
initializer = tf.keras.initializers.Orthogonal()

#Adding 4 layes of CNN.
models.add(layers.Conv2D(64, kernel_size=(5, 1), activation="relu",padding='same',kernel_initializer=initializer))
#model.add(BatchNormalization())
#model.add(Dropout(0.2))
models.add(layers.Conv2D(64, kernel_size=(5, 1), activation="relu",padding='same',kernel_initializer=initializer))
#model.add(BatchNormalization())
#model.add(Dropout(0.2))
models.add(layers.Conv2D(64, kernel_size=(5, 1), activation="relu",padding='same',kernel_initializer=initializer))
models.add(layers.Conv2D(64, kernel_size=(5, 1), activation="relu",padding='same',kernel_initializer=initializer))
models.add(layers.Permute((1,3,2)))
models.add(layers.Reshape( (int(models.layers[4].output_shape[1]), int(models.layers[4].output_shape[2]) * int(models.layers[4].output_shape[3]))))

#Adding LSTM layers.
models.add(layers.LSTM(128,dropout=0.2,return_sequences=True,kernel_initializer=initializer))
#model.add(Activation("relu"))
models.add(layers.LSTM(64,dropout=0.2,return_sequences=True,kernel_initializer=initializer))
models.add(Activation("relu"))
models.add(layers.Flatten())

#Applying dense layer of Softmax to the output of 4-CNN layers and 2 LSTM layers.
models.add(Dense(64, activation='sigmoid'))
models.add(layers.Dense(y.shape[1], activation="softmax"))
#Printing Summery of Model.
models.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 20, 2, 64)         384       
                                                                 
 conv2d_9 (Conv2D)           (None, 20, 2, 64)         20544     
                                                                 
 conv2d_10 (Conv2D)          (None, 20, 2, 64)         20544     
                                                                 
 conv2d_11 (Conv2D)          (None, 20, 2, 64)         20544     
                                                                 
 permute_2 (Permute)         (None, 20, 64, 2)         0         
                                                                 
 reshape_2 (Reshape)         (None, 20, 128)           0         
                                                                 
 lstm_4 (LSTM)               (None, 20, 128)          

In [ ]:
#tf.keras.Sequential([tf.keras.Input(shape=(50,2)), tf.keras.layers.Dense(3) ])
#tf.keras.Sequential([tf.keras.Input(shape=(1,)), tf.keras.layers.Dense(1) ])



In [ ]:
#model.compile(loss = tf.keras.losses.mae, 
              #optimizer = tf.keras.optimizers.SGD(), 
              #metrics = ["mae"])
#model.fit(x, y, epochs = 5)

In [ ]:
#model.compile(loss='categorical_crossentropy', optimizer="adam",metrics=['accuracy'])


In [ ]:
from sklearn.model_selection import train_test_split
com_train, com_test, y_train, y_test =train_test_split(com, Y, test_size=0.30)

ValueError: ignored

In [ ]:
models.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.CategoricalAccuracy()])


In [ ]:
models.fit(com_train, y_train,batch_size=20, epochs=20
           ,validation_data=(com_test,y_test)
             
              )

In [ ]:
y1_pred = models.predict(com_test)
print(confusion_matrix(y_test.argmax(axis=1), y1_pred.argmax(axis=1)))
print(classification_report(y_test.argmax(axis=1), y1_pred.argmax(axis=1)))
CM=(confusion_matrix(y_test.argmax(axis=1), y1_pred.argmax(axis=1)))
print("LSTMCONV2D Accuracy: {:.2f}".format(accuracy_score(y_test.argmax(axis=1), y1_pred.argmax(axis=1))))
print("Cohen's Kappa {:.2f}".format(cohen_kappa_score(y_test.argmax(axis=1), y1_pred.argmax(axis=1))))